In [2]:
!pip install -q crewai langchain langchain_core langchain_community langchain-google-genai langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.0/173.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.0/397.0 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 21.5 MB/s eta

In [34]:
# schemas.py

from pydantic import BaseModel, Field
from typing import List, Optional, Dict

class CodeInput(BaseModel):
    code_snippet: str
    language: str = Field(default="python", description="Programming language of the code snippet")
    context: Optional[str] = Field(default=None, description="Additional context or comments about the code")

class IssueDetail(BaseModel):
    issue_id: int
    description: str
    line_number: Optional[int]
    severity: str
    suggestion: Optional[str]

class AnalysisOutput(BaseModel):
    issues: List[IssueDetail]
    complexity_score: float
    cyclomatic_complexity: Optional[int]
    maintainability_index: Optional[float]
    technical_debt: Optional[float]

class RefactoringOpportunity(BaseModel):
    opportunity_id: int
    description: str
    related_issues: List[int]
    priority: str

class RefactoringOpportunities(BaseModel):
    opportunities: List[RefactoringOpportunity]

class SuggestionDetail(BaseModel):
    opportunity_id: int
    suggestion: str
    estimated_effort_hours: Optional[float]
    dependencies_affected: Optional[List[str]]

class RefactoringSuggestions(BaseModel):
    suggestions: List[SuggestionDetail]

class RefactoredCode(BaseModel):
    code_snippet: str
    changes_made: Dict[str, str]
    new_dependencies: Optional[List[str]]

In [4]:
from google.colab import userdata
import os
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [5]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [6]:
!pip install -q langchain-experimental wikipedia "wikibase-rest-api-client<0.2" mediawikiapi arxiv

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.6/103.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.6 MB/s eta 0:00:00


In [35]:
# agents.py

from crewai import Agent
from textwrap import dedent
from langchain_openai import ChatOpenAI
from langchain_experimental.tools import PythonREPLTool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools.wikidata.tool import WikidataAPIWrapper, WikidataQueryRun
from langchain_community.utilities import ArxivAPIWrapper
from langchain.tools import Tool

class CustomAgents:
    def __init__(self):
        self.OpenAIGPT4Mini = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        self.OpenAIGPT4 = ChatOpenAI(model="gpt-4o", temperature=0)

    def analysis_agent(self):
        # Agent 1: Code Analysis Expert
        tools = [PythonREPLTool()]
        return Agent(
            role="Code Analysis Expert",
            backstory=dedent("""You are an expert in analyzing code to detect issues, bugs, code smells, and provide complexity metrics."""),
            goal=dedent("""Analyze the provided code and identify any issues, potential bugs, or code smells. Also, provide complexity metrics such as cyclomatic complexity, maintainability index, and technical debt."""),
            tools=tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4Mini,
        )

    def opportunity_agent(self):
        # Agent 2: Refactoring Opportunity Identifier
        tools = [WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper()),
                 WikidataQueryRun(api_wrapper=WikidataAPIWrapper()),
                 Tool( # Wrap ArxivAPIWrapper as a Tool
                     name="Arxiv",
                     func=ArxivAPIWrapper().run,
                     description="A wrapper around Arxiv. Useful for when you need to access academic papers."
                 )]
        return Agent(
            role="Refactoring Opportunity Identifier",
            backstory=dedent("""You specialize in identifying specific opportunities for code refactoring based on analysis results."""),
            goal=dedent("""Identify specific refactoring opportunities based on the analysis output, relating them to the identified issues, and assign a priority level."""),
            tools=tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4,
        )

    def suggestion_agent(self):
        # Agent 3: Refactoring Suggestions Expert
        tools = [WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper()),
                 WikidataQueryRun(api_wrapper=WikidataAPIWrapper()),
                 Tool( # Wrap ArxivAPIWrapper as a Tool
                     name="Arxiv",
                     func=ArxivAPIWrapper().run,
                     description="A wrapper around Arxiv. Useful for when you need to access academic papers."
                 )]
        return Agent(
            role="Refactoring Suggestions Expert",
            backstory=dedent("""You provide detailed suggestions on how to implement the identified refactoring opportunities, including estimated effort and affected dependencies."""),
            goal=dedent("""Generate detailed suggestions for implementing the refactoring opportunities, including estimated effort in hours and any affected dependencies."""),
            tools=tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4Mini,
        )

    def refactoring_agent(self):
        # Agent 4: Code Refactoring Specialist
        tools = [PythonREPLTool()]
        return Agent(
            role="Code Refactoring Specialist",
            backstory=dedent("""You apply refactoring suggestions to the code and produce the refactored code along with a summary of changes made and any new dependencies."""),
            goal=dedent("""Apply the refactoring suggestions to the code and produce the refactored code along with a summary of the changes made and any new dependencies introduced."""),
            tools=tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4,
        )


In [36]:
# tasks.py

from crewai import Task
from textwrap import dedent

class CustomTasks:
    def __init__(self):
        pass

    def code_analysis_task(self, agent, code_input: CodeInput):
        analysis_output_schema = AnalysisOutput.schema_json(indent=2)
        return Task(
            description=dedent(f"""
                Analyze the following {code_input.language} code snippet and identify any issues, potential bugs, or code smells.
                Also, provide complexity metrics such as cyclomatic complexity, maintainability index, and technical debt.
                Provide your output in **JSON format** matching the **AnalysisOutput** schema.

                **Format**:
                ```json
                {analysis_output_schema}
                ```

                **Code**:
                ```{code_input.language}
                {code_input.code_snippet}
                ```

                **Additional Context**:
                {code_input.context if code_input.context else 'N/A'}
            """),
            agent=agent,
            expected_output=f"The analysis output in JSON format matching the schema: {analysis_output_schema}",
        )

    def refactoring_opportunity_task(self, agent, code_input: CodeInput):
        opportunities_schema = RefactoringOpportunities.schema_json(indent=2)
        return Task(
            description=dedent(f"""
                Based on the analysis of the following code, identify specific refactoring opportunities.
                Relate each opportunity to potential issues and assign a priority level (High, Medium, Low).
                Provide your output in **JSON format** matching the **RefactoringOpportunities** schema.

                **Format**:
                ```json
                {opportunities_schema}
                ```

                **Code**:
                ```{code_input.language}
                {code_input.code_snippet}
                ```

                **Additional Context**:
                {code_input.context if code_input.context else 'N/A'}
            """),
            agent=agent,
            expected_output=f"The refactoring opportunities in JSON format matching the schema: {opportunities_schema}",
        )

    def refactoring_suggestion_task(self, agent, code_input: CodeInput):
        refactoring_suggestions_schema = RefactoringSuggestions.schema_json(indent=2)
        return Task(
            description=dedent(f"""
                Generate detailed suggestions for implementing refactoring opportunities in the following code.
                Include estimated effort in hours for each suggestion.
                Provide your output in **JSON format** matching the **RefactoringSuggestions** schema.

                **Format**:
                ```json
                {refactoring_suggestions_schema}
                ```

                **Code**:
                ```{code_input.language}
                {code_input.code_snippet}
                ```

                **Additional Context**:
                {code_input.context if code_input.context else 'N/A'}
            """),
            agent=agent,
            expected_output=f"The refactoring suggestions in JSON format matching the schema: {refactoring_suggestions_schema}",
        )

    def code_refactoring_task(self, agent, code_input: CodeInput):
        refactored_code_schema = RefactoredCode.schema_json(indent=2)
        return Task(
            description=dedent(f"""
                Apply refactoring suggestions to the following code.
                Ensure the code remains functional and follows best practices.
                Provide the refactored code and a summary of changes made in **JSON format** matching the **RefactoredCode** schema.

                **Format**:
                ```json
                {refactored_code_schema}
                ```

                **Original Code**:
                ```{code_input.language}
                {code_input.code_snippet}
                ```

                **Additional Context**:
                {code_input.context if code_input.context else 'N/A'}
            """),
            agent=agent,
            expected_output=f"The refactored code in JSON format matching the schema: {refactored_code_schema}",
        )

In [37]:
# main.py

import os
from crewai import Crew
from textwrap import dedent
import json

class CodeRefactoringCrew:
    def __init__(self, code_snippet, language="python", context=None):
        self.code_input = CodeInput(code_snippet=code_snippet, language=language, context=context)
        self.agents = CustomAgents()
        self.tasks = CustomTasks()

    def run(self):
        # Define agents
        analysis_agent = self.agents.analysis_agent()
        opportunity_agent = self.agents.opportunity_agent()
        suggestion_agent = self.agents.suggestion_agent()
        refactoring_agent = self.agents.refactoring_agent()

        # Define tasks
        analysis_task = self.tasks.code_analysis_task(analysis_agent, self.code_input)
        opportunity_task = self.tasks.refactoring_opportunity_task(opportunity_agent, self.code_input)
        suggestion_task = self.tasks.refactoring_suggestion_task(suggestion_agent, self.code_input)
        refactoring_task = self.tasks.code_refactoring_task(refactoring_agent, self.code_input)

        # Create the crew
        crew = Crew(
            agents=[
                analysis_agent,
                opportunity_agent,
                suggestion_agent,
                refactoring_agent,
            ],
            tasks=[
                analysis_task,
                opportunity_task,
                suggestion_task,
                refactoring_task,
            ],
            verbose=True,
        )

        result = crew.kickoff()
        return result

if __name__ == "__main__":
    print("## Welcome to the Code Refactoring Assistant")
    print("-------------------------------------------------------")
    code_snippet = input(dedent("""Please enter the code you wish to refactor:\n"""))
    language = input("Enter the programming language (default is Python): ") or "python"
    context = input("Any additional context or comments about the code? ") or None

    crew = CodeRefactoringCrew(code_snippet, language, context)
    results = crew.run()
    print("\n\n########################")
    print("## Here is you custom crew run result:")
    print("########################\n")
    print(results)

## Welcome to the Code Refactoring Assistant
-------------------------------------------------------
Please enter the code you wish to refactor:
    def quicksort(arr):         if len(arr) == 1:             return arr # This will fail for an empty array, causing a crash         pivot = arr[0]         left = []         right = []         for i in arr:             if i < pivot:                 left.append(pivot)             else:                 right.append(pivot)         return quicksort(left) + quicksort([pivot]) + quicksort(right)      arr = [3, 6, 8, 10, 1, 2, 1]     sorted_arr = quicksort(arr)     print(sorted_arr)
Enter the programming language (default is Python): python
Any additional context or comments about the code? This is an incorrect Quicksort algorithm that needs major refactoring and fixes.


# Agent: Code Analysis Expert
## Task: 
                Analyze the following python code snippet and identify any issues, potential bugs, or code smells.
                Also, provide complexity metrics such as cyclomatic complexity, maintainability index, and technical debt.
                Provide your output in **JSON format** matching the **AnalysisOutput** schema.

                **Format**:
                ```json
                {
  "$defs": {
    "IssueDetail": {
      "properties": {
        "issue_id": {
          "title": "Issue Id",
          "type": "integer"
        },
        "description": {
          "title": "Description",
          "type": "string"
        },
        "line_number": {
          "anyOf": [
            {
              "type": "integer"
            },
            {
              "type": "null"
            }
          ],
          "title": "Line Number"
        },
        "severity": {
          "title": "Severity",
          "type": "string"
        },

In [39]:
results

CrewOutput(raw='```json\n{\n  "code_snippet": "def quicksort(arr):\\n    if len(arr) <= 1:\\n        return arr  # Corrected to handle empty arrays as well\\n    pivot = arr[0]\\n    left = [x for x in arr[1:] if x < pivot]  # Corrected logic and used list comprehension\\n    right = [x for x in arr[1:] if x >= pivot]  # Corrected logic and used list comprehension\\n    return quicksort(left) + [pivot] + quicksort(right)\\n\\narr = [3, 6, 8, 10, 1, 2, 1]\\nsorted_arr = quicksort(arr)\\nprint(sorted_arr)\\n",\n  "changes_made": {\n    "1": "Added a check for an empty array at the beginning of the function to prevent potential crashes.",\n    "2": "Corrected the pivot selection and appending logic to ensure elements are correctly added to the left and right lists.",\n    "3": "Used list comprehensions to improve code readability and efficiency."\n  },\n  "new_dependencies": null\n}\n```', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='\n                Analyze the f